In [26]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

def collate_gpu(batch):
    x, t = torch.utils.data.default_collate(batch)
    return x.to(device="cuda:0"), t.to(device="cuda:0")

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

train_dataloader = DataLoader(training_data, batch_size=64, collate_fn=collate_gpu)
test_dataloader = DataLoader(test_data, batch_size=64, collate_fn=collate_gpu)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to('cuda')

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [24]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [27]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.297291  [   64/60000]
loss: 2.287467  [ 6464/60000]
loss: 2.267522  [12864/60000]
loss: 2.266227  [19264/60000]
loss: 2.247244  [25664/60000]
loss: 2.206251  [32064/60000]
loss: 2.227251  [38464/60000]
loss: 2.187495  [44864/60000]
loss: 2.185482  [51264/60000]
loss: 2.155483  [57664/60000]
Test Error: 
 Accuracy: 41.2%, Avg loss: 2.152923 

Epoch 2
-------------------------------
loss: 2.160769  [   64/60000]
loss: 2.151549  [ 6464/60000]
loss: 2.099595  [12864/60000]
loss: 2.122250  [19264/60000]
loss: 2.068611  [25664/60000]
loss: 2.003375  [32064/60000]
loss: 2.042011  [38464/60000]
loss: 1.962958  [44864/60000]
loss: 1.969868  [51264/60000]
loss: 1.902194  [57664/60000]
Test Error: 
 Accuracy: 59.1%, Avg loss: 1.899889 

Epoch 3
-------------------------------
loss: 1.930698  [   64/60000]
loss: 1.898835  [ 6464/60000]
loss: 1.796539  [12864/60000]
loss: 1.839409  [19264/60000]
loss: 1.721159  [25664/60000]
loss: 1.674147  [32064/600